# **This notebook handle Feature Engineering**

## Objectives

* The objective of this notebook is to perform feature engineering on the healthcare diabetes dataset to prepare it for predictive modeling. 
* This includes creating new features, transforming existing ones, handling outliers, scaling .

## Inputs

* The input to this notebook is cleaned input file (cleaned_diabetes_data.csv) located in the data/cleaned directory.

## Outputs

* This noteo=book will produce two output files located in the data/transformed directory.

    * **diabetes_dashboard.csv**   - for Power BI dashboard visualizations.
    * **diabetes_model_input.csv** - for ML model training and evaluation.


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\nived\\Desktop\\Nivya work learnings\\data analytics_ai\\Capstone\\Healthcare_Diabetes_Analysis\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\nived\\Desktop\\Nivya work learnings\\data analytics_ai\\Capstone\\Healthcare_Diabetes_Analysis'

# Features addition

* Here we will add new features based on the EDA findings, group and transform existing features.

* Import necessary libraries

In [27]:
import pandas as pd
import numpy as np
from feature_engine.outliers import Winsorizer
from feature_engine.encoding import OneHotEncoder
from sklearn.pipeline import Pipeline

* Load the cleaned input file into a dataframe and display the first few rows

In [28]:
df = pd.read_csv('Data/cleaned/cleaned_diabetes_data.csv')
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Male,80,0,0,former,26.40,8.2,126,1
1,Female,47,0,0,never,45.88,4.0,159,0
2,Female,26,0,0,not current,27.32,6.0,130,0
3,Male,80,0,0,No Info,27.32,7.5,160,1
4,Male,57,0,0,No Info,27.32,5.8,158,0


* Bin the BMI into categories as per WHO classification.
* I plan to use this column for dashboard visualizations and let the model use the actual BMI values.

In [51]:
# Define BMI bins based on WHO classification
bmi_bins = [0, 18.5, 25, 30, float('inf')]

# Define the labels for each range
bmi_labels = ['underweight', 'normal', 'overweight', 'obesity']

# Bin the data using pd.cut()
df['bmi_category'] = pd.cut(df['bmi'], bins=bmi_bins, labels=bmi_labels)

# checking for the value counts of the new bmi category
print(df['bmi_category'].value_counts())

# display the first few rows to verify the new column
df.head()

bmi_category
overweight     4126
obesity        3479
normal         1356
underweight      81
Name: count, dtype: int64


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes,bmi_category,age_group,diabetes_risk,smoking_binned
0,Male,80,0,0,former,26.40,8.2,126,1,overweight,Seniors (61+),Very High Risk,smoker
1,Female,47,0,0,never,45.88,4.0,159,0,obesity,Late Middle Age (46–60),Moderate Risk,non-smoker
2,Female,26,0,0,not current,27.32,6.0,130,0,overweight,Young Adults (18–30),Moderate Risk,smoker
3,Male,80,0,0,No Info,27.32,7.5,160,1,overweight,Seniors (61+),Very High Risk,unknown
4,Male,57,0,0,No Info,27.32,5.8,158,0,overweight,Late Middle Age (46–60),High Risk,unknown


* Below is the mapping for BMI categories:
  - Underweight: BMI < 18.5
  - Normal weight: 18.5 ≤ BMI ≤ 24.9
  - Overweight: 25 ≤ BMI ≤ 29.9
  - Obesity: BMI ≥ 30

* Grouping age into bins for better visualization in dashboard.

In [52]:
# Define age bins into 4 categories
age_bins = [18, 30, 45, 60, 100]

# Define the labels for each range
age_labels = ['Young Adults (18–30)', 
              'Early Middle Age (31–45)', 
              'Late Middle Age (46–60)', 
              'Seniors (61+)']

# Bin the data using pd.cut()
df['age_group'] = pd.cut(
    df['age'],
    bins=age_bins,
    labels=age_labels,
    right=True,          # includes the right edge, so 30 is in first bin, 45 in second, etc.
    include_lowest=True  # ensures 18 is included in the first bin
)

# checking for the value counts of the new bmi category
print(df['age_group'].value_counts())

# display the first few rows to verify the new column
df.head()

age_group
Seniors (61+)               3757
Late Middle Age (46–60)     2627
Early Middle Age (31–45)    1645
Young Adults (18–30)        1013
Name: count, dtype: int64


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes,bmi_category,age_group,diabetes_risk,smoking_binned
0,Male,80,0,0,former,26.40,8.2,126,1,overweight,Seniors (61+),Very High Risk,smoker
1,Female,47,0,0,never,45.88,4.0,159,0,obesity,Late Middle Age (46–60),Moderate Risk,non-smoker
2,Female,26,0,0,not current,27.32,6.0,130,0,overweight,Young Adults (18–30),Moderate Risk,smoker
3,Male,80,0,0,No Info,27.32,7.5,160,1,overweight,Seniors (61+),Very High Risk,unknown
4,Male,57,0,0,No Info,27.32,5.8,158,0,overweight,Late Middle Age (46–60),High Risk,unknown


* Creating a new feature called 'diabetes_risk' based on HbA1c levels and blood glucose levels.
* This feature will categorize patients into 'Low Risk', 'Pre-Diabetes', and 'High Risk (Diabetes)' based on their HbA1c and blood glucose levels.
  - Low Risk: HbA1c < 5.7 or Blood Glucose < 140
  - Moderate Risk: 5.7 ≤ HbA1c < 6.5 or 140 ≤ Blood Glucose < 200
  - High Risk: 5.7 ≤ HbA1c < 6.5 and 140 ≤ Blood Glucose < 200
  - Very High Risk (Diabetes): HbA1c ≥ 6.5 or Blood Glucose ≥ 200

In [53]:
risk_conditions = [
    (df['HbA1c_level'] >= 6.5) | (df['blood_glucose_level'] >= 200),
    (df['HbA1c_level'].between(5.7, 6.4)) & (df['blood_glucose_level'].between(140, 199)),
    (df['HbA1c_level'].between(5.7, 6.4)) | (df['blood_glucose_level'].between(140, 199)),
    (df['HbA1c_level'] < 5.7) | (df['blood_glucose_level'] < 140)
]

risk_labels = [
    'Very High Risk',
    'High Risk',
    'Moderate Risk',
    'Low Risk'
]

df['diabetes_risk'] = np.select(risk_conditions, risk_labels)

print(df['diabetes_risk'].value_counts())

# display the first few rows to verify the new column
df.head()

diabetes_risk
Very High Risk    4836
Moderate Risk     1941
High Risk         1496
Low Risk           769
Name: count, dtype: int64


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes,bmi_category,age_group,diabetes_risk,smoking_binned
0,Male,80,0,0,former,26.40,8.2,126,1,overweight,Seniors (61+),Very High Risk,smoker
1,Female,47,0,0,never,45.88,4.0,159,0,obesity,Late Middle Age (46–60),Moderate Risk,non-smoker
2,Female,26,0,0,not current,27.32,6.0,130,0,overweight,Young Adults (18–30),Moderate Risk,smoker
3,Male,80,0,0,No Info,27.32,7.5,160,1,overweight,Seniors (61+),Very High Risk,unknown
4,Male,57,0,0,No Info,27.32,5.8,158,0,overweight,Late Middle Age (46–60),High Risk,unknown


* Grouping the smoking history into just 3 categories - 'never','smoker','unknown'.
* This is to reduce the number of categories and simplify the understanding of the data when used in dashboard.

In [54]:
# grouping the smoking history
def bin_smoking(value):
    if value == 'never':
        return 'non-smoker'
    elif value in ['current', 'former', 'ever', 'not current']:
        return 'smoker'
    else:
        return 'unknown'

df['smoking_binned'] = df['smoking_history'].apply(bin_smoking)

# checking the value counts of the actual smoking history column and the new binned column
print(f"value counts smoking before: ",df['smoking_history'].value_counts())
print(f"value counts smoking after: ",df['smoking_binned'].value_counts())

# display the first few rows to verify the new column
df.head()

value counts smoking before:  smoking_history
never          3534
No Info        2026
former         1352
current         988
not current     685
ever            457
Name: count, dtype: int64
value counts smoking after:  smoking_binned
non-smoker    3534
smoker        3482
unknown       2026
Name: count, dtype: int64


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes,bmi_category,age_group,diabetes_risk,smoking_binned
0,Male,80,0,0,former,26.40,8.2,126,1,overweight,Seniors (61+),Very High Risk,smoker
1,Female,47,0,0,never,45.88,4.0,159,0,obesity,Late Middle Age (46–60),Moderate Risk,non-smoker
2,Female,26,0,0,not current,27.32,6.0,130,0,overweight,Young Adults (18–30),Moderate Risk,smoker
3,Male,80,0,0,No Info,27.32,7.5,160,1,overweight,Seniors (61+),Very High Risk,unknown
4,Male,57,0,0,No Info,27.32,5.8,158,0,overweight,Late Middle Age (46–60),High Risk,unknown


* Save this dataframe to a new df_dashboard. We will later save this dataframe as a csv file to be used in dashboard visualizations.

In [55]:
df_dashboard = df.copy()
df_dashboard.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes,bmi_category,age_group,diabetes_risk,smoking_binned
0,Male,80,0,0,former,26.40,8.2,126,1,overweight,Seniors (61+),Very High Risk,smoker
1,Female,47,0,0,never,45.88,4.0,159,0,obesity,Late Middle Age (46–60),Moderate Risk,non-smoker
2,Female,26,0,0,not current,27.32,6.0,130,0,overweight,Young Adults (18–30),Moderate Risk,smoker
3,Male,80,0,0,No Info,27.32,7.5,160,1,overweight,Seniors (61+),Very High Risk,unknown
4,Male,57,0,0,No Info,27.32,5.8,158,0,overweight,Late Middle Age (46–60),High Risk,unknown


* **One-hot encode categorical variables**: Convert categorical variables(gender and smoking_binned) into a format that can be provided to ML algorithms to do a better job in prediction.

* Use **Winsorization** to handle outliers in bmi and HbA1c columns as observed in EDA notebook.

In [56]:
pipeline = Pipeline([
      ( 'ohe_smoker', OneHotEncoder(variables=['smoking_binned','gender'],drop_last=True)),
      ( 'winsorizer_iqr', Winsorizer(capping_method='iqr', fold=1.5, tail='both', variables=['bmi', 'HbA1c_level']) )
])

df_transformed = pipeline.fit_transform(df)
df_transformed.head()

,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes,bmi_category,age_group,diabetes_risk,smoking_binned_smoker,smoking_binned_non-smoker,gender_Male
0,80,0,0,former,26.40,8.20,126,1,overweight,Seniors (61+),Very High Risk,1,0,1
1,47,0,0,never,43.10,4.05,159,0,obesity,Late Middle Age (46–60),Moderate Risk,0,1,0
2,26,0,0,not current,27.32,6.00,130,0,overweight,Young Adults (18–30),Moderate Risk,1,0,0
3,80,0,0,No Info,27.32,7.50,160,1,overweight,Seniors (61+),Very High Risk,0,0,1
4,57,0,0,No Info,27.32,5.80,158,0,overweight,Late Middle Age (46–60),High Risk,0,0,1


* Next will select only those columns from the transformed dataframe that are needed for ML modelling.

In [58]:
# checking the info of the transformed dataframe, to see the list of columns

df_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9042 entries, 0 to 9041
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   age                        9042 non-null   int64   
 1   hypertension               9042 non-null   int64   
 2   heart_disease              9042 non-null   int64   
 3   smoking_history            9042 non-null   object  
 4   bmi                        9042 non-null   float64 
 5   HbA1c_level                9042 non-null   float64 
 6   blood_glucose_level        9042 non-null   int64   
 7   diabetes                   9042 non-null   int64   
 8   bmi_category               9042 non-null   category
 9   age_group                  9042 non-null   category
 10  diabetes_risk              9042 non-null   object  
 11  smoking_binned_smoker      9042 non-null   int32   
 12  smoking_binned_non-smoker  9042 non-null   int32   
 13  gender_Male                9042 n

In [59]:
# selecting only the needed columns for ML modelling and loading it into a new dataframe

columns_to_keep = [
    'age',
    'bmi',
    'HbA1c_level',
    'blood_glucose_level',
    'hypertension',
    'heart_disease',
    'smoking_binned_smoker',
    'smoking_binned_non-smoker',
    'gender_Male',
    'diabetes'  # Target column
]

df_ml_model = df_transformed[columns_to_keep].copy()

# checking the first few rows of the final dataframe for ML modelling
df_ml_model.head()

,age,bmi,HbA1c_level,blood_glucose_level,hypertension,heart_disease,smoking_binned_smoker,smoking_binned_non-smoker,gender_Male,diabetes
0,80,26.40,8.20,126,0,0,1,0,1,1
1,47,43.10,4.05,159,0,0,0,1,0,0
2,26,27.32,6.00,130,0,0,1,0,0,0
3,80,27.32,7.50,160,0,0,0,0,1,1
4,57,27.32,5.80,158,0,0,0,0,1,0


---


* **Save the 2 new dataframes df_dashboard and df_ml_model to csv files in the transformed data folder.**

In [60]:
current_dir = os.getcwd()
current_dir

df_dashboard.to_csv('Data/transformed/diabetes_dashboard.csv', index=False)
df_ml_model.to_csv('Data/transformed/diabetes_model_input.csv', index=False)


---

### Conclusion and Next Steps

* The feature engineering process has been completed, resulting in two new dataframes: df_dashboard and df_ml_model.
* The df_dashboard dataframe is tailored for visualization purposes, while the df_ml_model dataframe is for ML modeling.
* Output CSV file - **diabetes_dashboard.csv** - will be used in Power BI dashboard visualizations.
* Output csv file - **diabetes_model_input.csv** - will be used in ML_Model notebook for ML model training and evaluation.